[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/camenduru/style-aligned-colab/blob/main/style_aligned_gradio_colab.ipynb)

In [ ]:
!pip install -q diffusers transformers accelerate peft gradio==3.50.2 mediapy einops
!pip install -q https://download.pytorch.org/whl/cu118/xformers-0.0.22.post4%2Bcu118-cp310-cp310-manylinux2014_x86_64.whl

%cd /content
!git clone -b dev https://github.com/camenduru/style-aligned
%cd /content/style-aligned

from diffusers import StableDiffusionXLPipeline, DDIMScheduler
import torch
import mediapy
import sa_handler

scheduler = DDIMScheduler(beta_start=0.00085, beta_end=0.012, beta_schedule="scaled_linear", clip_sample=False, set_alpha_to_one=False)
pipeline = StableDiffusionXLPipeline.from_pretrained("stabilityai/stable-diffusion-xl-base-1.0", torch_dtype=torch.float16, variant="fp16", use_safetensors=False, scheduler=scheduler).to("cuda")
# pipeline.enable_xformers_memory_efficient_attention()
handler = sa_handler.Handler(pipeline)
sa_args = sa_handler.StyleAlignedArgs(share_group_norm=False, share_layer_norm=False, share_attention=True, adain_queries=True, adain_keys=True, adain_values=False)
handler.register(sa_args)

import gradio as gr

def generate(promp1, promp2, negative_prompt, num_inference_steps, width, height):
    sets_of_prompts = [promp1, promp2]
    images = pipeline(prompt=sets_of_prompts, negative_prompt=negative_prompt, width=width, height=height, num_inference_steps=num_inference_steps).images
    return images

with gr.Blocks(title=f"Realtime SDXL Turbo", css=".gradio-container {max-width: 850px !important}") as demo:
    with gr.Group():
      prompt1 = gr.Textbox(show_label=False, value="a toy train. macro photo. 3d game asset")
      prompt2 = gr.Textbox(show_label=False, value="a toy airplane. macro photo. 3d game asset")
      with gr.Row():
          negative_prompt = gr.Textbox(show_label=False, value="low-resolution")
          button = gr.Button()
      with gr.Row():
          width = gr.Slider(minimum=256, maximum=2048, step=8, value=768, label="width")
          height = gr.Slider(minimum=256, maximum=2048, step=8, value=768, label="height")
          num_inference_steps = gr.Slider(minimum=1, maximum=150, step=1, value=50, label="num_inference_steps")
      output_images = gr.Gallery(
          show_label=False,
          type="pil",
          interactive=False,
          height=816,
          width=816,
          elem_id="output_image",
      )
    button.click(fn=generate, inputs=[prompt1, prompt2, negative_prompt, num_inference_steps, width, height], outputs=[output_images], show_progress=True)
demo.queue().launch(inline=False, share=True, debug=True)